# 1. Get Data

In [4]:
# pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tweepy  # https://github.com/tweepy/tweepy
import csv
import json
import sys
import datetime
import matplotlib.pyplot as plt
from pytz import timezone
import hashlib
import pandas as pd

In [6]:
# CREDENTIALS
api_key = "zGWacA8CkRVNQgY5vgAh4nfAp"
api_secret_key = "DWaRZpAls3u4YcotLKcImKeYy03qlcBqXBKlDr6CtK5Y33Jc3I"
access_token = "1140211942436429825-5D5hkMjOGr5WyxciM8qO7WH6lz0cOq"
access_token_secret = "AZZ0OhhV2t8W1n9Q8MqopgtvZaChXsZzxwr1CbF9OmcIz"

auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [4]:
# Total 33 companied of DJIA
companies = "3M,Walgreens,Salesforce,Amgen,Honeywell,American Express,Apple,Boeing,Caterpillar,Chevron,Cisco Systems," \
            "Coca-Cola,ExxonMobil,General Electric,Goldman Sachs,IBM," \
            "Intel,Johnson & Johnson,JPMorgan Chase,McDonald's,Merck,Microsoft,Nike," \
            "Pfizer,Procter & Gamble,The Home Depot,Travelers,United Technologies," \
            "UnitedHealth Group,Verizon,Visa,Walmart,Walt Disney".split(",")

accounts = "3M,Walgreens,Salesforce,Amgen,Honeywell,AmericanExpress,AppleSupport,Boeing,CaterpillarInc,Chevron,Cisco," \
           "CocaCola,exxonmobil,generalelectric,GoldmanSachs,IBM,intel," \
           "JNJNews,jpmorgan,McDonalds,Merck,Microsoft,Nike,pfizer,ProcterGamble," \
           "HomeDepot,Travelers,UTC,UnitedHealthGrp,verizon,Visa,Walmart,DisneyStudios".split(",")

comDic = dict(zip(accounts, companies))

In [5]:
comDic

{'3M': '3M',
 'Walgreens': 'Walgreens',
 'Salesforce': 'Salesforce',
 'Amgen': 'Amgen',
 'Honeywell': 'Honeywell',
 'AmericanExpress': 'American Express',
 'AppleSupport': 'Apple',
 'Boeing': 'Boeing',
 'CaterpillarInc': 'Caterpillar',
 'Chevron': 'Chevron',
 'Cisco': 'Cisco Systems',
 'CocaCola': 'Coca-Cola',
 'exxonmobil': 'ExxonMobil',
 'generalelectric': 'General Electric',
 'GoldmanSachs': 'Goldman Sachs',
 'IBM': 'IBM',
 'intel': 'Intel',
 'JNJNews': 'Johnson & Johnson',
 'jpmorgan': 'JPMorgan Chase',
 'McDonalds': "McDonald's",
 'Merck': 'Merck',
 'Microsoft': 'Microsoft',
 'Nike': 'Nike',
 'pfizer': 'Pfizer',
 'ProcterGamble': 'Procter & Gamble',
 'HomeDepot': 'The Home Depot',
 'Travelers': 'Travelers',
 'UTC': 'United Technologies',
 'UnitedHealthGrp': 'UnitedHealth Group',
 'verizon': 'Verizon',
 'Visa': 'Visa',
 'Walmart': 'Walmart',
 'DisneyStudios': 'Walt Disney'}

In [6]:
# function to get the tweets base on their screen_name(user_id)
def get_all_tweets(screen_name):
    # Twitter only allows access to a users most recent 3200 tweets with this method

    print("collecting tweets from: " + screen_name)
    # initialize a list to hold all the tweepy Tweets
    alltweets = []

    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name=screen_name, count=200)

    # save most recent tweets
    alltweets.extend(new_tweets)

    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        # all subsequent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name, count=200,
                                       max_id=oldest, exclude_replies=True)

        # save most recent tweets
        alltweets.extend(new_tweets)

        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    # transform the tweepy tweets into a 2D array that will populate the csv
    outtweets = [[
        tweet.created_at,
        comDic[screen_name],
        "https://twitter.com/" + screen_name + "/status/" + tweet.id_str,
        tweet.text.encode("utf-8"),
        tweet.retweet_count,
        tweet.favorite_count,
        tweet.id_str]
        for tweet in alltweets]

    # compose dataframe for outputting
    df = pd.DataFrame(columns=["created_at", "company", "url", "tweets", "re_tweets", "likes", "id"])
    for tweet in outtweets:
        row = pd.Series(tweet,index=["created_at", "company", "url", "tweets", "re_tweets", "likes", "id"])
        df = df.append(row, ignore_index=True)
        
    return df

In [7]:
allData_list = []
for acc in accounts:
    allData_list.append(get_all_tweets(acc))

collecting tweets from: 3M
...257 tweets downloaded so far
...297 tweets downloaded so far
...356 tweets downloaded so far
...409 tweets downloaded so far
...467 tweets downloaded so far
...537 tweets downloaded so far
...578 tweets downloaded so far
...622 tweets downloaded so far
...682 tweets downloaded so far
...731 tweets downloaded so far
...753 tweets downloaded so far
...770 tweets downloaded so far
...803 tweets downloaded so far
...835 tweets downloaded so far
...867 tweets downloaded so far
...894 tweets downloaded so far
...894 tweets downloaded so far
collecting tweets from: Walgreens
...207 tweets downloaded so far
...210 tweets downloaded so far
...212 tweets downloaded so far
...213 tweets downloaded so far
...214 tweets downloaded so far
...215 tweets downloaded so far
...217 tweets downloaded so far
...228 tweets downloaded so far
...232 tweets downloaded so far
...239 tweets downloaded so far
...249 tweets downloaded so far
...253 tweets downloaded so far
...260 twee

...2107 tweets downloaded so far
...2306 tweets downloaded so far
...2506 tweets downloaded so far
...2706 tweets downloaded so far
...2906 tweets downloaded so far
...3106 tweets downloaded so far
...3156 tweets downloaded so far
...3156 tweets downloaded so far
collecting tweets from: IBM
...219 tweets downloaded so far
...233 tweets downloaded so far
...256 tweets downloaded so far
...280 tweets downloaded so far
...292 tweets downloaded so far
...297 tweets downloaded so far
...306 tweets downloaded so far
...321 tweets downloaded so far
...330 tweets downloaded so far
...338 tweets downloaded so far
...349 tweets downloaded so far
...360 tweets downloaded so far
...373 tweets downloaded so far
...383 tweets downloaded so far
...398 tweets downloaded so far
...409 tweets downloaded so far
...409 tweets downloaded so far
collecting tweets from: intel
...335 tweets downloaded so far
...440 tweets downloaded so far
...521 tweets downloaded so far
...572 tweets downloaded so far
...636

...730 tweets downloaded so far
...746 tweets downloaded so far
...854 tweets downloaded so far
...949 tweets downloaded so far
...1038 tweets downloaded so far
...1117 tweets downloaded so far
...1192 tweets downloaded so far
...1232 tweets downloaded so far
...1232 tweets downloaded so far
collecting tweets from: Visa
...235 tweets downloaded so far
...253 tweets downloaded so far
...257 tweets downloaded so far
...258 tweets downloaded so far
...259 tweets downloaded so far
...261 tweets downloaded so far
...262 tweets downloaded so far
...263 tweets downloaded so far
...330 tweets downloaded so far
...330 tweets downloaded so far
collecting tweets from: Walmart
...204 tweets downloaded so far
...208 tweets downloaded so far
...210 tweets downloaded so far
...236 tweets downloaded so far
...237 tweets downloaded so far
...239 tweets downloaded so far
...241 tweets downloaded so far
...242 tweets downloaded so far
...244 tweets downloaded so far
...249 tweets downloaded so far
...250

In [8]:
allData_df = pd.concat(allData_list).reset_index(drop=True)
allData_df

,created_at,company,url,tweets,re_tweets,likes,id
0,2022-08-16 18:35:08+00:00,3M,https://twitter.com/3M/status/1559609719840440320,b'@bob0pearls Hello - We are sorry to hear of ...,0,0,1559609719840440320
1,2022-08-16 17:44:04+00:00,3M,https://twitter.com/3M/status/1559596870866984963,"b'To help increase visibility and safety, this...",0,10,1559596870866984963
2,2022-08-16 17:04:41+00:00,3M,https://twitter.com/3M/status/1559586957969293312,b'@__deeannaa Hello - We are sorry to hear of ...,0,0,1559586957969293312
3,2022-08-16 16:48:00+00:00,3M,https://twitter.com/3M/status/1559582761865846785,b'@DeptofRage Hello - We are sorry to hear of ...,0,0,1559582761865846785
4,2022-08-15 21:15:12+00:00,3M,https://twitter.com/3M/status/1559287617908244482,b'@thicciceberg Hello - We are sorry to hear o...,0,2,1559287617908244482
...,...,...,...,...,...,...,...
55196,2019-04-07 19:06:00+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11149...,"b'In Dreamland, everything is possible. See #D...",104,937,1114967577883762691
55197,2019-04-07 16:00:00+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11149...,b'Celebrate 10 years of @Disneynature in advan...,100,734,1114920771753844736
55198,2019-04-06 19:00:00+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11146...,"b'See the #1 movie in the world, in theaters n...",51,522,1114603682195095553
55199,2019-04-06 18:00:01+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11145...,b'There\xe2\x80\x99s no one quite like Steve. ...,207,1340,1114588585485918208


In [9]:
allData_df['tweets'][0]

b'@bob0pearls Hello - We are sorry to hear of the trouble! Please connect with our Command Brand product support team\xe2\x80\xa6 https://t.co/9AhLbxI8ut'

In [10]:
allData_df.describe()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,created_at,company,url,tweets,re_tweets,likes,id
count,55201,55201,55201,55201,55201.0,55201.0,55201
unique,54771,33,55201,55062,1020.0,2038.0,55201
top,2022-08-08 14:18:14+00:00,Goldman Sachs,https://twitter.com/3M/status/1559609719840440320,b'Here are 5 ways Amgen supports the environme...,0.0,0.0,1559609719840440320
freq,4,3156,1,10,5572.0,12178.0,1
first,2012-05-17 17:22:38+00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,2022-08-17 01:15:16+00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# allData_df.to_pickle("Merge_data.pkl")

In [4]:
start_date = "2022-04-15"
start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end_date = "2022-07-15"
end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

# Right way!
start_date = start_date.replace(tzinfo=timezone('UTC'))
end_date = end_date.replace(tzinfo=timezone('UTC'))

In [13]:
print(start_date)
print(end_date)

2022-05-01 00:00:00+00:00
2022-07-01 00:00:00+00:00


In [15]:
prune_tweets_data = allData_df.query(' created_at >= @start_date and created_at <= @end_date').reset_index(drop=True)
prune_tweets_data = prune_tweets_data.sort_values(by="created_at").reset_index(drop=True)
prune_tweets_data

,created_at,company,url,tweets,re_tweets,likes,id
0,2022-05-01 12:00:47+00:00,Caterpillar,https://twitter.com/CaterpillarInc/status/1520...,b'Find out how our customized operator trainin...,2,33,1520734977914200064
1,2022-05-01 12:08:13+00:00,Cisco Systems,https://twitter.com/Cisco/status/1520736848938...,"b""While the day for when we'll have a powerful...",6,24,1520736848938688513
2,2022-05-01 13:53:45+00:00,Nike,https://twitter.com/Nike/status/15207634062355...,b'It was never about what she got from basketb...,2206,7958,1520763406235541504
3,2022-05-01 15:00:01+00:00,Verizon,https://twitter.com/verizon/status/15207800846...,"b'When the pandemic hit, Principal Akbar Cook\...",3,23,1520780084658929664
4,2022-05-01 15:00:25+00:00,Salesforce,https://twitter.com/Salesforce/status/15207801...,b'Is it time to ditch the resume?\n\nAs an emp...,10,29,1520780184655478784
...,...,...,...,...,...,...,...
3083,2022-06-30 23:00:31+00:00,The Home Depot,https://twitter.com/HomeDepot/status/154264427...,"b'After getting their hands a little dirty, cr...",8,38,1542644277687320576
3084,2022-06-30 23:08:11+00:00,Intel,https://twitter.com/intel/status/1542646206144...,"b""@magius2013 @Alienware https://t.co/VYF2Vq76...",0,2,1542646206144208897
3085,2022-06-30 23:26:35+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/15426...,"b""RT @Disney_Insiders: It's time to rise toget...",11,0,1542650836899155969
3086,2022-06-30 23:31:00+00:00,American Express,https://twitter.com/AmericanExpress/status/154...,"b'@odieloveslattes Hi Danae, so sorry to hear ...",0,0,1542651949870465026


In [ ]:
# prune_tweets_data.to_pickle("MayToJune_TweetsData.pkl")

## more data
update_date: 0915<br>
reference: https://github.com/mahrtayyab/tweety<br>
goal: By searching a keyword to scrape<br>
description: Get 20 Tweets for a specific Keyword or Hashtag

- param wait_time : (`int`) seconds to wait between multiple requests

In [4]:
from tweety.bot import Twitter
trends = Twitter().search("DJIA", pages = 500, wait_time = 2)
trends.to_csv("0916.csv")

In [7]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.read_csv("0916.csv")

,Created on,author,is_retweet,is_reply,tweet_id,tweet_body,language,likes,retweet_count,source,medias,user_mentioned,urls,hashtags,symbols
0,Thu Sep 15 19:28:33 +0000 2022,BisphamGreen,False,False,1570494798968287232,"In 2056 when the DJIA finally retakes 36000, w...",en,9.0,NaN,Twitter for Android,NaN,NaN,NaN,NaN,NaN
1,Thu Sep 15 16:28:43 +0000 2022,Anupam Kaulshi,False,False,1570449543619293185,"#Nifty ratio with Dow #DJIA at all time high, ...",en,3.0,1.0,Twitter Web App,https://twitter.com/alittleabtmkt/status/1570...,NaN,NaN,Nifty; DJIA; nifty50;,NaN
2,Wed Sep 14 06:42:29 +0000 2022,Mayur Shah | Polestar Capital,False,False,1569939626223349760,Whn u do ur research well &amp; have strong co...,en,21.0,5.0,Twitter Web App,NaN,NaN,NaN,nifty50; NSE; StockMarket;,NaN
3,Thu Sep 15 15:58:16 +0000 2022,$,False,False,1570441879535374339,$DJIA H4 $YM_F #DowJones \nThis count is based...,en,28.0,4.0,Twitter Web App,https://twitter.com/JoeChartingStar/status/15...,NaN,NaN,DowJones;,"[{'text': 'DJIA', 'indices': [0, 5]}, {'text':..."
4,Fri Sep 16 00:15:38 +0000 2022,"FiboSwanny, Chartered Financial Technician - CFTe",False,False,1570567045418463233,Fun with some Fib levels on the DJIA Monthly C...,en,NaN,NaN,Twitter for iPhone,NaN,tradingview;,NaN,DJI;,NaN
5,Tue Sep 13 12:36:10 +0000 2022,Liz Ann Sonders,False,False,1569666246999678979,Major issue beneath headline: food component o...,en,1602.0,692.0,Twitter for iPad,https://twitter.com/LizAnnSonders/status/1569...,NaN,NaN,inflation;,NaN
6,Tue Sep 13 12:43:03 +0000 2022,David Asman,False,False,1569667979352104962,Whoops! White House was expecting to celebrate...,en,510.0,282.0,Twitter for iPhone,NaN,NaN,NaN,NaN,NaN
7,Tue Sep 13 16:07:21 +0000 2022,#Nuclear Cup OCovfefe,False,True,1569719390265802758,@JoeBiden Is this why the family net worth dro...,en,86.0,6.0,Twitter for iPhone,NaN,JoeBiden;,NaN,NaN,NaN
8,Fri Sep 09 15:46:27 +0000 2022,RUDRA,False,False,1568264581822451712,#SNP #NASDAQ #DowJones #DJIA \nLETS SEE FOR RE...,en,1.0,NaN,Twitter Web App,https://twitter.com/RUDRA_ANSH_SHIV/status/15...,NaN,NaN,SNP; NASDAQ; DowJones; DJIA;,NaN
9,Wed Sep 14 06:09:46 +0000 2022,Gary Glover,False,False,1569931391370149888,I sent this chart out on Monday to my clients....,en,17.0,2.0,Twitter Web App,https://twitter.com/GloverGary/status/1569931...,NaN,NaN,NaN,"[{'text': 'DJIA', 'indices': [85, 90]}]"


In [ ]:
trends.to_csv("0915.csv")
# pd.set_option("display.max_rows", None)
# pd.read_csv("0915.csv")

# 2. Preprocess

## I. removal of stop-words and punctuation<br>
First thing first. To make natural language understandable for machines.
So here are four things to do:

1. load data
2. preprocess the sentences

refer: http://www.lrec-conf.org/proceedings/lrec2014/pdf/292_Paper.pdf

In [ ]:
# pip install nltk

In [16]:
# 如果你想往前移再往前拿
from nltk.corpus import stopwords
import re

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/lala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
EngStopWords = set(stopwords.words('english'))

In [19]:
def preprocess(text): 
    text = re.sub(r'[^\w\s]',' ',text)
    
    tokens = []
    for word in text.split():
        if word in EngStopWords:
            pass #如果詞彙是個英文的停用詞的話，就略過不處理。
        else:
            tokens.append(word)
    return tokens


In [20]:
tryyy = prune_tweets_data['tweets'][0]
# preprocess(tryyy)
tryyy

b'Find out how our customized operator training is helping the team at Morgan Corp. improve productivity, enhance saf\xe2\x80\xa6 https://t.co/uphkCfP3nq'

In [21]:
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1\1", s)

#start process_tweet
def processTweet(tweet):
    # process the tweets
    
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)    
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

#start getfeatureVector
def getFeatureVector(tweet, stopWords):
    featureVector = []  
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences 
        w = replaceTwoOrMore(w) 
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if it consists of only words
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*[a-zA-Z]+[a-zA-Z0-9]*$", w)
        #ignore if it is a stopWord
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector   

#start extract_features
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return 


In [22]:
count = 0;
featureList = []
tweets = []

In [ ]:
tryyy

In [24]:
tryyy = str(tryyy)
tweet = tryyy
processedTweet = processTweet(tweet)
featureVector = getFeatureVector(processedTweet, EngStopWords)
# featureList.extend(featureVector)
# tweets.append((featureVector, sentiment))

In [25]:
print(featureList)
print(tweets)

['customized', 'operator', 'training', 'helping', 'team', 'morgan', 'corp', 'improve', 'productivity', 'enhance', 'url']
[]


In [71]:
prune_tweets_data.head()

,created_at,company,url,tweets,re_tweets,likes,id
0,2022-05-01 12:00:47+00:00,Caterpillar,https://twitter.com/CaterpillarInc/status/1520...,b'Find out how our customized operator trainin...,2,33,1520734977914200064
1,2022-05-01 12:08:13+00:00,Cisco Systems,https://twitter.com/Cisco/status/1520736848938...,"b""While the day for when we'll have a powerful...",6,24,1520736848938688513
2,2022-05-01 13:53:45+00:00,Nike,https://twitter.com/Nike/status/15207634062355...,b'It was never about what she got from basketb...,2206,7958,1520763406235541504
3,2022-05-01 15:00:01+00:00,Verizon,https://twitter.com/verizon/status/15207800846...,"b'When the pandemic hit, Principal Akbar Cook\...",3,23,1520780084658929664
4,2022-05-01 15:00:25+00:00,Salesforce,https://twitter.com/Salesforce/status/15207801...,b'Is it time to ditch the resume?\n\nAs an emp...,10,29,1520780184655478784


In [72]:
distil_df = pd.DataFrame(prune_tweets_data, columns=['created_at','tweets'])
# 轉換created_at
distil_df ['created_at'] = distil_df ['created_at'].astype('str')
distil_df['date'] = pd.to_datetime( distil_df['created_at'], format="%Y-%m-%d %H:%M:%S+00:00")
distil_df = distil_df.sort_values(by=['date'], ignore_index = True)

# 轉換tweets
distil_df ['tweets'] = distil_df ['tweets'].astype('str')

del distil_df ['created_at'] 
distil_df.head()

,tweets,date
0,b'Find out how our customized operator trainin...,2022-05-01 12:00:47
1,"b""While the day for when we'll have a powerful...",2022-05-01 12:08:13
2,b'It was never about what she got from basketb...,2022-05-01 13:53:45
3,"b'When the pandemic hit, Principal Akbar Cook\...",2022-05-01 15:00:01
4,b'Is it time to ditch the resume?\n\nAs an emp...,2022-05-01 15:00:25


In [73]:
distil_df.describe()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,tweets,date
count,3088,3088
unique,3067,3046
top,b'Here are 5 ways Amgen supports the environme...,2022-06-23 14:14:57
freq,8,3
first,NaN,2022-05-01 12:00:47
last,NaN,2022-06-30 23:37:39


In [74]:
def preprocess(text):
    tweet = str(text)
    processedTweet = processTweet(tweet)
    featureVector = getFeatureVector(processedTweet, EngStopWords)
    return featureVector 

In [75]:
distil_df['tweets']

0       b'Find out how our customized operator trainin...
1       b"While the day for when we'll have a powerful...
2       b'It was never about what she got from basketb...
3       b'When the pandemic hit, Principal Akbar Cook\...
4       b'Is it time to ditch the resume?\n\nAs an emp...
                              ...                        
3083    b'After getting their hands a little dirty, cr...
3084    b"@magius2013 @Alienware https://t.co/VYF2Vq76...
3085    b"RT @Disney_Insiders: It's time to rise toget...
3086    b'@odieloveslattes Hi Danae, so sorry to hear ...
3087    b'RT @MicrosoftLife: \xe2\x80\x9cWe have far m...
Name: tweets, Length: 3088, dtype: object

In [76]:
distil_df['tweets_processed'] = distil_df.apply(lambda x: preprocess(x['tweets']),axis=1)
del distil_df['tweets']
distil_df

,date,tweets_processed
0,2022-05-01 12:00:47,"[customized, operator, training, helping, team..."
1,2022-05-01 12:08:13,"[day, powerful, quantumcomputer, still, tbd, s..."
2,2022-05-01 13:53:45,"[never, got, basketball, always, gave, back, g..."
3,2022-05-01 15:00:01,"[pandemic, hit, principal, akbar, school, beca..."
4,2022-05-01 15:00:25,"[time, ditch, employer, need, question, rethin..."
...,...,...
3083,2022-06-30 23:00:31,"[getting, hands, little, dirty, creators, dyla..."
3084,2022-06-30 23:08:11,"[url, july, welcome]"
3085,2022-06-30 23:26:35,"[time, rise, talking, director, akin, omotoso,..."
3086,2022-06-30 23:31:00,"[hi, danae, sorry, hear, looping, case, help]"


### GPOMS data

In [50]:
def processTweet_G(tweet):

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)    
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

def preprocess_GPOMS(text):
    tweet = str(text)
    processedTweet = processTweet_G(tweet)
#     featureVector = getFeatureVector(processedTweet, EngStopWords)
    return processedTweet

In [51]:
GPOMS_df['tweets_processed'][0]

"b'find out how our customized operator training is helping the team at morgan corp. improve productivity, enhance saf\\xe2\\x80\\xa6 URL"

In [58]:
distil_df

,date,tweet_size
0,2022-05-01 12:00:47,133
1,2022-05-01 12:08:13,127
2,2022-05-01 13:53:45,133
3,2022-05-01 15:00:01,144
4,2022-05-01 15:00:25,135
...,...,...
3083,2022-06-30 23:00:31,141
3084,2022-06-30 23:08:11,74
3085,2022-06-30 23:26:35,157
3086,2022-06-30 23:31:00,90


In [64]:
GPOMS_df = pd.DataFrame(prune_tweets_data, columns=['created_at','tweets'])
GPOMS_df['tweets_processed'] = GPOMS_df.apply(lambda x: preprocess_GPOMS(x['tweets']),axis=1)
del GPOMS_df['tweets']
GPOMS_df

,created_at,tweets_processed
0,2022-05-01 12:00:47+00:00,b'find out how our customized operator trainin...
1,2022-05-01 12:08:13+00:00,"b""while the day for when we'll have a powerful..."
2,2022-05-01 13:53:45+00:00,b'it was never about what she got from basketb...
3,2022-05-01 15:00:01+00:00,"b'when the pandemic hit, principal akbar cook\..."
4,2022-05-01 15:00:25+00:00,b'is it time to ditch the resume?\n\nas an emp...
...,...,...
3083,2022-06-30 23:00:31+00:00,"b'after getting their hands a little dirty, cr..."
3084,2022-06-30 23:08:11+00:00,"b"" have until july 7th. you're welcome \xf0\x9..."
3085,2022-06-30 23:26:35+00:00,"b""rt it's time to rise together! \xf0\x9f\x8f\..."
3086,2022-06-30 23:31:00+00:00,"b' hi danae, so sorry to hear this. looping in..."


In [65]:
GPOMS_df['tweet_size'] = GPOMS_df['tweets_processed'].map(len)
GPOMS_df

,created_at,tweets_processed,tweet_size
0,2022-05-01 12:00:47+00:00,b'find out how our customized operator trainin...,130
1,2022-05-01 12:08:13+00:00,"b""while the day for when we'll have a powerful...",124
2,2022-05-01 13:53:45+00:00,b'it was never about what she got from basketb...,130
3,2022-05-01 15:00:01+00:00,"b'when the pandemic hit, principal akbar cook\...",141
4,2022-05-01 15:00:25+00:00,b'is it time to ditch the resume?\n\nas an emp...,132
...,...,...,...
3083,2022-06-30 23:00:31+00:00,"b'after getting their hands a little dirty, cr...",130
3084,2022-06-30 23:08:11+00:00,"b"" have until july 7th. you're welcome \xf0\x9...",55
3085,2022-06-30 23:26:35+00:00,"b""rt it's time to rise together! \xf0\x9f\x8f\...",149
3086,2022-06-30 23:31:00+00:00,"b' hi danae, so sorry to hear this. looping in...",75


In [67]:
GPOMS_df.describe()

,tweet_size
count,3088.000000
mean,123.701425
std,32.318110
min,1.000000
25%,114.000000
50%,129.000000
75%,140.250000
max,345.000000


In [70]:
GPOMS_df.to_pickle("corpus_GPOMS.pkl")

## II. group all tweets that were submitted on the same date

In [77]:
distil_df['date_processed'] = distil_df.apply(lambda x: datetime.datetime.strftime(x['date'], "%Y-%m-%d"),axis=1)
del distil_df['date'] 
distil_df

,tweets_processed,date_processed
0,"[customized, operator, training, helping, team...",2022-05-01
1,"[day, powerful, quantumcomputer, still, tbd, s...",2022-05-01
2,"[never, got, basketball, always, gave, back, g...",2022-05-01
3,"[pandemic, hit, principal, akbar, school, beca...",2022-05-01
4,"[time, ditch, employer, need, question, rethin...",2022-05-01
...,...,...
3083,"[getting, hands, little, dirty, creators, dyla...",2022-06-30
3084,"[url, july, welcome]",2022-06-30
3085,"[time, rise, talking, director, akin, omotoso,...",2022-06-30
3086,"[hi, danae, sorry, hear, looping, case, help]",2022-06-30


In [78]:
sector = distil_df.groupby("date_processed")
sector.size()

date_processed
2022-05-01    15
2022-05-02    47
2022-05-03    47
2022-05-04    50
2022-05-05    54
              ..
2022-06-26     8
2022-06-27    47
2022-06-28    50
2022-06-29    61
2022-06-30    80
Length: 61, dtype: int64

In [79]:
condense_df = distil_df.groupby("date_processed").agg({'tweets_processed':'sum'})
condense_df

,tweets_processed
date_processed,
2022-05-01,"[customized, operator, training, helping, team..."
2022-05-02,"[power, higher, education, one, network, time,..."
2022-05-03,"[ambulances, make, india, bharti, airtel, hosp..."
2022-05-04,"[cisco, calculates, emissions, products, math,..."
2022-05-05,"[employees, said, able, work, remotely, made, ..."
...,...
2022-06-26,"[follow, final, day, action, sundays, trophies..."
2022-06-27,"[hola, enviarnos, un, mensaje, privado, desde,..."
2022-06-28,"[ciscolive, keynote, roundup, las, ante, learn..."


In [80]:
condense_df['tweet_size'] = condense_df['tweets_processed'].map(len)
condense_df

,tweets_processed,tweet_size
date_processed,,
2022-05-01,"[customized, operator, training, helping, team...",142
2022-05-02,"[power, higher, education, one, network, time,...",413
2022-05-03,"[ambulances, make, india, bharti, airtel, hosp...",425
2022-05-04,"[cisco, calculates, emissions, products, math,...",438
2022-05-05,"[employees, said, able, work, remotely, made, ...",474
...,...,...
2022-06-26,"[follow, final, day, action, sundays, trophies...",54
2022-06-27,"[hola, enviarnos, un, mensaje, privado, desde,...",459
2022-06-28,"[ciscolive, keynote, roundup, las, ante, learn...",444


In [81]:
# pip install plotly

In [82]:
import plotly.express as px

fig = px.line(condense_df, y="tweet_size", title='Variation of tweet size')
fig.show()

In [83]:
condense_df.describe()

,tweet_size
count,61.000000
mean,437.245902
std,258.296061
min,54.000000
25%,158.000000
50%,459.000000
75%,602.000000
max,1119.000000


In [84]:
condense_df.to_pickle("corpus.pkl")

## III. only take into account tweets that contain explicit statements of their author’s mood states 

In [ ]:
# 再塞下去怕詞太少捏～～

## IV.  filter out tweets that match the regular expressions “http:” or “www(avoid spam messages and other information-oriented tweets)

In [172]:
# 已經在preprocess拿掉ㄌ！ Good!

##  Another: 取得yfinance → DJIA 財金資料 to dataframe2

In [80]:
# pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
import pandas as pd

In [84]:
start_date

datetime.datetime(2022, 5, 1, 0, 0, tzinfo=<UTC>)

In [2]:
def get_historical_data(ticker):
    d = yf.Ticker(ticker)
    df = d.history(period='max')
#     df = d.history(start="2022-06-01", end="2022-06-30", interval="1m")
#     df = d.history(start=start_date, end=end_date)
    df.columns = df.columns.str.lower()
    df.columns = pd.Series(df.columns).str.capitalize().values
    
    return df.dropna()

In [3]:
df = get_historical_data('DJI')
df

,Open,High,Low,Close,Volume,Dividends,Stock splits
Date,,,,,,,
1970-01-02,809.200012,809.200012,809.200012,809.200012,0,0,0
1970-01-05,811.309998,811.309998,811.309998,811.309998,0,0,0
1970-01-06,803.659973,803.659973,803.659973,803.659973,0,0,0
1970-01-07,801.809998,801.809998,801.809998,801.809998,0,0,0
1970-01-08,802.070007,802.070007,802.070007,802.070007,0,0,0
...,...,...,...,...,...,...,...
2022-02-24,33223.800781,33223.800781,33223.800781,33223.800781,0,0,0
2022-02-25,34058.800781,34058.800781,34058.800781,34058.800781,0,0,0
2022-02-28,33892.601562,33892.601562,33892.601562,33892.601562,0,0,0


沒有我們要的區間：） <br>
改用以下的方法

In [87]:
# pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pandas_datareader import data, wb
DJIA = data.DataReader(name='^DJI', data_source='yahoo',start=start_date, end=end_date)
DJIA 

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-04-14,34889.171875,34437.500000,34628.460938,34451.230469,353680000,34451.230469
2022-04-18,34618.289062,34279.078125,34411.488281,34411.691406,264110000,34411.691406
2022-04-19,34983.109375,34394.621094,34394.621094,34911.199219,296350000,34911.199219
2022-04-20,35315.398438,34962.671875,34962.671875,35160.789062,337620000,35160.789062
2022-04-21,35492.218750,34723.820312,35258.800781,34792.761719,366390000,34792.761719
...,...,...,...,...,...,...
2022-07-11,31367.550781,31114.400391,31277.980469,31173.839844,245020000,31173.839844
2022-07-12,31346.099609,30860.509766,31113.310547,30981.330078,324190000,30981.330078
2022-07-13,30979.169922,30514.609375,30743.630859,30772.789062,291830000,30772.789062


In [6]:
DJIA.to_csv("DJIA_price.csv")